In [1]:
import psycopg2
import pandas as pd
import plotly.express as px
import requests
import json

In [12]:
#sql connection
mydb=psycopg2.connect(host="localhost",
                      user="postgres",
                      port="5432",
                      database="phonepe_data",
                      password="Ramraji")
cursor=mydb.cursor()

#aggregated insurance dataframe
cursor.execute("SELECT * FROM aggregrated_insurance")
mydb.commit()
table1=cursor.fetchall()

aggregrated_insurance=pd.DataFrame(table1,columns=("States","Years","Quarters","Transaction_type","Transaction_count","Transaction_amount"))

#aggregated transaction dataframe
cursor.execute("SELECT * FROM aggregrated_transaction")
mydb.commit()
table2=cursor.fetchall()

aggregrated_transaction=pd.DataFrame(table2,columns=("States","Years","Quarters","Transaction_type","Transaction_count","Transaction_amount"))

#aggregated user dataframe
cursor.execute("SELECT * FROM aggregrated_user")
mydb.commit()
table3=cursor.fetchall()

aggregrated_user=pd.DataFrame(table3,columns=("States","Years","Quarters","Brands","Transaction_count","Percentage"))

#map insurance dataframe
cursor.execute("SELECT * FROM map_insurance")
mydb.commit()
table4=cursor.fetchall()

map_insurance=pd.DataFrame(table4,columns=("States","Years","Quarters","Districts","Transaction_count","Transaction_amount"))

#map transaction dataframe
cursor.execute("SELECT * FROM map_transaction")
mydb.commit()
table5=cursor.fetchall()

map_transaction=pd.DataFrame(table5,columns=("States","Years","Quarters","Districts","Transaction_count","Transaction_amount"))

#map user dataframe
cursor.execute("SELECT * FROM map_user")
mydb.commit()
table6=cursor.fetchall()

map_user=pd.DataFrame(table6,columns=("States","Years","Quarters","Districts","RegisteredUsers","AppOpens"))


#top insurance dataframe
cursor.execute("SELECT * FROM top_insurance")
mydb.commit()
table7=cursor.fetchall()

top_insurance=pd.DataFrame(table7,columns=("States","Years","Quarters","Pincodes","Transaction_count","Transaction_amount"))


#top transaction dataframe
cursor.execute("SELECT * FROM top_transaction")
mydb.commit()
table8=cursor.fetchall()

top_transaction=pd.DataFrame(table8,columns=("States","Years","Quarters","Pincodes","Transaction_count","Transaction_amount"))


#top user dataframe
cursor.execute("SELECT * FROM top_user")
mydb.commit()
table9=cursor.fetchall()

top_user=pd.DataFrame(table9,columns=("States","Years","Quarters","Pincodes","RegisteredUsers"))

In [ ]:
url="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
response=requests.get(url)
data1=json.loads(response.content)
states_name=[]
for feature in data1["features"]:
    states_name.append(feature["properties"]["ST_NM"])

states_name.sort()

fig_india_1=px.choropleth(tacyg,geojson=data1,locations="States",featureidkey="properties.ST_NM",
                          color="Transaction_amount",color_continuous_scale="Rainbow",range_color=(tacyg["Transaction_amount"].min(),tacyg["Transaction_amount"].max()),
                          hover_name="States",title=f"{year} TRANSACTION AMOUNT",fitbounds="locations",
                          height=650,weight=600)
fig_india_1.update_goes(visible=False)
fig_india_1.show()

fig_india_2=px.choropleth(tacyg,geojson=data1,locations="States",featureidkey="properties.ST_NM",
                          color="Transaction_count",color_continuous_scale="Rainbow",range_color=(tacyg["Transaction_count"].min(),tacyg["Transaction_count"].max()),
                          hover_name="States",title=f"{year} TRANSACTION COUNT",fitbounds="locations",
                          height=650,weight=600)
fig_india_2.update_goes(visible=False)
fig_india_2.show()

In [ ]:
def Agg_trans_Trans_type(df,state):
   tacy=df[df["States"]==state]
   tacy.reset_index(drop=True,inplace=True)

   tacyg=tacy.groupby("Transaction_type")[["Transaction_count","Transaction_amount"]].sum()
   tacyg.reset_index(inplace=True)

   fig_pie_1=px.pie(data_frame=tacyg,names="Transaction_type",values="Transaction_amount",width=600,title=f"{state.upper()} TRANSACTION AMOUNT",hole=0.5)

   fig_pie_2=px.pie(data_frame=tacyg,names="Transaction_type",values="Transaction_count",width=600,title=f"{state.upper()} TRANSACTION COUNT",hole=0.5)

In [ ]:
def Aggr_user_year(df,year):
    aguy=df[df["Years"]==year]
    aguy.reset_index(drop=True,inplace=True)

    aguyg=pd.DataFrame(aguy.groupby("Brands")["Transaction_count"].sum())
    aguyg.reset_index(inplace=True)

    fig_bar_1=px.bar(aguyg,x="Brands",y="Transaction_count",title="BRANDS AND TRANSACTION COUNT",width=1000,color_discrete_sequence=px.colors.sequential.haline)
    st.plotly_chart(fig_bar_1)

In [ ]:
def top_chart_transaction_amount(table_name):
#sql connection
    mydb=psycopg2.connect(host="localhost",
                        user="postgres",
                        port="5432",
                        database="phonepe_data",
                        password="Ramraji")
    cursor=mydb.cursor()

    #Question 1 query:
    query1=f'''SELECT states,SUM(transaction_amount) AS transaction_amount
            FROM {table_name}
            GROUP BY states
            ORDER BY transaction_amount DESC
            LIMIT 10'''

    cursor.execute(query1)
    table_1=cursor.fetchall()
    mydb.commit()

    df_1=pd.DataFrame(table_1,columns=("states","transaction_amount"))

    fig_bar_1=px.bar(df_1,x="states",y="transaction_amount",title="STATES AND TRANSACTION AMOUNT",hover_name="states",color_discrete_sequence=px.colors.sequential.Agsunset,height=650,width=600)
    st.plotly_chart(fig_bar_1)

    #Question 2 query:
    query2=f'''SELECT states,SUM(transaction_amount) AS transaction_amount
            FROM {table_name}
            GROUP BY states
            ORDER BY transaction_amount 
            LIMIT 10'''

    cursor.execute(query2)
    table_2=cursor.fetchall()
    mydb.commit()

    df_2=pd.DataFrame(table_2,columns=("states","transaction_amount"))

    fig_bar_2=px.bar(df_2,x="states",y="transaction_amount",title="STATES AND TRANSACTION AMOUNT",hover_name="states",color_discrete_sequence=px.colors.sequential.Agsunset_r,height=650,width=600)
    st.plotly_chart(fig_bar_2)

    #Question 3 query:
    query3=f'''SELECT states,AVG(transaction_amount) AS transaction_amount
            FROM FROM {table_name}
            GROUP BY states
            ORDER BY transaction_amount'''

    cursor.execute(query3)
    table_3=cursor.fetchall()
    mydb.commit()

    df_3=pd.DataFrame(table_3,columns=("states","transaction_amount"))

    fig_bar_3=px.bar(df_3,x="transaction_amount",y="states",orientation="h",title="STATES AND TRANSACTION AMOUNT",hover_name="states",color_discrete_sequence=px.colors.sequential.Blues,height=650,width=600)
    st.plotly_chart(fig_bar_3)